In [14]:
import os
import configparser
import urllib
import csv
import datetime
import sys
import requests
import json
import pandas
from bs4 import BeautifulSoup
import numpy as np
import glob
#from forex_python.converter import CurrencyRates
import re

In [15]:
currentdir = 'c:\\Users\\303435\\OneDrive - The Warehouse Group\\Desktop\\Temp\\Data\\MAX\\py_venv\\PriceCheckerV2'
search_list = currentdir+'\SearchList.txt'
#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36' #Change to different agent if 403 error code return, it may the server/website identify you are webscrapping 
#user_agent = 'python-requests/2.28.1'
#user_agent = 'Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
headers={'User-Agent':user_agent} 
url_template = 'https://cdkeyprices.com/search?sk={name}&categories=&platforms=all_xbox&types=game&filter=1' 
extract_file_name = 'Price_List.csv'
result_datetime = datetime.datetime.now(tz=None).strftime("%d%m%Y_%H%M")

last_week_date_raw = datetime.datetime.today() + datetime.timedelta(days=-7)
last_week_date = last_week_date_raw.strftime("%d-%m-%Y") 
last_week_files = []
files = glob.iglob(currentdir + '\\file\\*', recursive=False) 
for i in files:
  i2 = i.split('\\')[-1]
  i3 = i2.split('_')[-2]
  if i3 > last_week_date:
      last_week_files.append(i2)


In [16]:
def sorting_file(last_week_files):
  last_week_files= last_week_files
  sort_order = []
  
  for i in last_week_files:
      i2 = datetime.datetime.strptime(i.split('_')[2].split('.')[0][0:2]+'-'+i.split('_')[2].split('.')[0][2:4]+'-'+i.split('_')[2].split('.')[0][4:], '%d-%m-%Y')
      sort_order.append(i2) 
  zipped = list(zip(last_week_files,sort_order))
  sorted_files = sorted(zipped, key = lambda x: x[1])
  sorted_files_trim = []
  for i in sorted_files:
    sorted_files_trim.append(i[0])
  return sorted_files_trim

In [17]:
def get_url_v2(search_list):
  name_list= []
  url_list = []
  f = open(search_list,"r")
  for i in f:
      if 'bought' in i:
          print('skip as '+i+ ' already')
          continue
      name = i.replace("'", "")
      n_name = name.replace("\n","")
      r_name = n_name.replace(" ","+")
      name_list.append(r_name)
  f.close()  
   
  for i in name_list:
    url = url_template.format(name = i)
    if url == 'https://cdkeyprices.com/item/dragon-quest-xi-echoes-of-an-elusive-age-cd-key-xbox':
      continue
    url_list.append(url)
  print('url list and name list are generated')       
  return url_list, name_list
 

In [18]:
def get_result(url_list):
  result_list = []
  price_usd_list = []
  captured_timestamp_list = []
  release_yr = []
  url = []
  for search_url in url_list:
    #request=urllib.request.Request(search_url,None,headers)
    #response = urllib.request.urlopen(request)
    #data = response.read()
    response = requests.get(search_url)
    if response.status_code ==200:
      data = response.text
      soup = BeautifulSoup(data,'html.parser')
      link = soup.findAll("div", {"class": "product-list-price buy_cell"})
      link_yr = soup.findAll("div", {"class": "product-list-year"})
    else:
      print(f"Failed to get data via {url_list}")
    
    for i in link_yr:
      release_yr.append(i.string)
    for i in link:
      result_name = i.contents[1]['href'].replace("https://cdkeyprices.com/item/","").replace   ("-cd-key-xbox","").replace("-"," ")
      t = i.contents[1]
      #print(t)
      if len(t) !=2:#in case there is a result return without price data
        print('skip '+(result_name.title())+ ' as no data returned')
        continue
      url.append(search_url)
      result_list.append((result_name.title()))
      t2 = t.contents[1]
      #price_usd_list.append(i.contents[1].contents[1]['data-price-usd-value'])
      price_usd_list.append(t2['data-price-usd-value'])
      captured_timestamp = datetime.datetime.now(tz=None)
      captured_timestamp_list.append(captured_timestamp.strftime("%x %X"))
      
  combined_list = list(zip( result_list,release_yr,price_usd_list,captured_timestamp_list ))
  return combined_list

In [19]:
#def get_currency(original_currency, converted_currency):
#  c = CurrencyRates()
#  exchange_rate = c.get_rate(original_currency, converted_currency)
#  return round(exchange_rate,2)

In [20]:
#There is a bug on forex api, so have to use it manually

fx_url = "https://www.freeforexapi.com/api/live?pairs=USDNZD"

payload = {}
headers = {}

response = requests.request("GET", fx_url, headers=headers, data=payload,verify=False)
response.status_code
response.text
x=json.loads(response.text)
USD_NZD = round(x['rates']['USDNZD']['rate'],2)
print('USD_NZD: '+str(USD_NZD))
 

c:\Users\303435\OneDrive - The Warehouse Group\Desktop\Temp\Data\MAX\py_venv\PriceCheckerV2\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.freeforexapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


USD_NZD: 1.62


In [21]:
def gen_comparison(combined_list, exchange_rate):
  price_data_new = pandas.DataFrame(combined_list, columns =[ 'result_name', 'release_yr','latest_captured_price_usd','latest_captured_timestamp', 'url' ])
  price_data_new = price_data_new.drop_duplicates(subset='result_name')

  #price_data_old = pandas.read_csv(currentdir+'\\'+ extract_file_name)
  last_week_file = sorting_file(last_week_files)

  try:
    price_data_last_week = pandas.read_csv(currentdir+'\\file\\'+ last_week_file[-7])
  except Exception: #Get the latest file found if there is no file 7days ago
    latest_file_num = len(last_week_files) * -1
    price_data_last_week = pandas.read_csv(currentdir+'\\file\\'+ last_week_file[latest_file_num])
  
  #Check if column is available before drop
  if 'last_week_price_usd' in price_data_last_week.columns:
    price_data_last_week = price_data_last_week.drop(['previous_price_usd','previous_captured_timestamp','last_week_price_usd','last_week_captured_timestamp'],axis=1)
  else:
    price_data_last_week = price_data_last_week.drop(['previous_price_usd','previous_captured_timestamp'],axis=1)

  price_data_last_week_renamed = price_data_last_week.rename( 
  columns ={
    "latest_captured_price_usd": "last_week_price_usd",
    "latest_captured_timestamp": "last_week_captured_timestamp"
})

  price_data_old = pandas.read_csv(currentdir+'\\'+ 'price_list_latest.csv')
  price_data_old = price_data_old.drop(['previous_price_usd','previous_captured_timestamp'],axis=1)
  price_data_old_renamed = price_data_old.rename( 
  columns ={
    "latest_captured_price_usd": "previous_price_usd",
    "latest_captured_timestamp": "previous_captured_timestamp"
})

  result = pandas.merge(price_data_new[['result_name','release_yr', 'latest_captured_price_usd','latest_captured_timestamp',  'url']],
                      price_data_old_renamed[['result_name', 'previous_price_usd','previous_captured_timestamp']],
                      on = 'result_name',
                      how ='left')
  result['latest_captured_price_usd'] = pandas.to_numeric(result['latest_captured_price_usd']).round(2)
  result['is_cheaper'] = np.where(result['latest_captured_price_usd']< result['previous_price_usd'],True,False)
  result['difference'] = (result['latest_captured_price_usd']-result['previous_price_usd']).round(2)

#ToDo change to weekly on Friday
  result_last_week = pandas.merge(result[['result_name','release_yr', 'latest_captured_price_usd','latest_captured_timestamp','is_cheaper','difference']],
                      price_data_last_week_renamed[['result_name', 'last_week_price_usd','last_week_captured_timestamp']],
                      on = 'result_name',
                      how ='left')
  result_last_week['latest_captured_price_usd'] = pandas.to_numeric(result_last_week['latest_captured_price_usd']).round(2)
  result_last_week['last_week_price_usd'] = pandas.to_numeric(result_last_week['last_week_price_usd']).round(2)
  result_last_week['is_cheaper_last_week'] = np.where(result_last_week['latest_captured_price_usd']< result_last_week['last_week_price_usd'],True,False)
  result_last_week['difference_last_week'] = (result_last_week['latest_captured_price_usd']-result_last_week['last_week_price_usd']).round(2)

  result_cheaper = result[result['difference'] <-1]
  result_cheaper_last_week = result_last_week[result_last_week['difference_last_week'] <-1]

 # result_cheaper[result_cheaper.select_dtypes(include=['number']).columns].round(2) *= exchange_rate
 # result_cheaper_last_week[result_cheaper_last_week.select_dtypes(include=['number']).columns] *= exchange_rate
  result_cheaper[result_cheaper.select_dtypes(include=['number']).columns] = result_cheaper[result_cheaper.select_dtypes(include=['number']).columns].round(2) * exchange_rate
  result_cheaper_last_week[result_cheaper_last_week.select_dtypes(include=['number']).columns] = result_cheaper_last_week[result_cheaper_last_week.select_dtypes(include=['number']).columns].round(2) * exchange_rate
    
  result.to_csv(currentdir+'\\file\\price_list_{result_datetime}.csv'.format(result_datetime = result_datetime),index=False)
  result.to_csv(currentdir+'\\price_list_latest.csv',index=False)  
  result_cheaper.to_csv(currentdir+'\\cheaper_price_list_latest.csv',index=False)

  result_last_week.to_csv(currentdir+'\\price_list_last_week.csv',index=False)  
  result_cheaper_last_week.to_csv(currentdir+'\\cheaper_price_list_last_week.csv',index=False)

  return result_cheaper, result_cheaper_last_week
 

In [22]:
#get_result_matched
def get_result_matched(url_list, name_list):
  result_list = []
  price_usd_list = []
  captured_timestamp_list = []
  release_yr = []
  url = []
  for search_url in url_list:
    response = requests.get(search_url)
    if response.status_code ==200:
      data = response.text
      soup = BeautifulSoup(data,'html.parser')
      link = soup.findAll("div", {"class": "product-list-price buy_cell"})
      link_yr = soup.findAll("div", {"class": "product-list-year"})
    else:
      print(f"Failed to get data via {url_list}")
    
    for i in link_yr:
      release_yr.append(i.string)
    for i in link:
      result_name = i.contents[1]['href'].replace("https://cdkeyprices.com/item/","").replace("-cd-key-xbox","").replace("-"," ")
      t = i.contents[1]
      #print(t)
      if len(t) !=2:#in case there is a result return without price data
        print('skip '+(result_name.title()) + ' as no data return')
        continue
      
      result_list.append((result_name.title()))
      t2 = t.contents[1]
      #price_usd_list.append(i.contents[1].contents[1]['data-price-usd-value'])
      price_usd_list.append(t2['data-price-usd-value'])
      url.append(search_url)
      captured_timestamp = datetime.datetime.now(tz=None)
      captured_timestamp_list.append(captured_timestamp.strftime("%x %X"))
      
  combined_list = list(zip(result_list, release_yr, price_usd_list, captured_timestamp_list, url))
  #combined_list = list(zip(result_list, release_yr, price_usd_list, captured_timestamp_list))
  
  filtered_list = []
  for name in name_list:
    clean_name = re.sub(r'[^a-zA-Z0-9]+', ' ', name)

    for res, yr, price, time, url in combined_list:
        if clean_name.lower() in res.lower():
            filtered_list.append((res, yr, price, time, url))    

  print('Search result is generated')                
  return filtered_list
 


In [23]:
#rename_columns_to_lower_case
import os
import csv

def rename_columns_to_lower_case():
    directory = currentdir+'\\file'
    # get all the CSV files in the directory
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    
    for file in csv_files:
        # create a new filename for the modified CSV
        new_filename = file.split('.')[0] + '_modified.csv'
        
        # open the input and output CSV files
        with open(os.path.join(directory, file), 'r') as input_file, open(os.path.join(directory, new_filename), 'w', newline='') as output_file:
            reader = csv.reader(input_file)
            writer = csv.writer(output_file)
            
            # read the header row and convert all column names to lower case
            header = next(reader)
            header = [column.lower() for column in header]
            
            # write the modified header to the output file
            writer.writerow(header)
            
            # copy the remaining rows from the input file to the output file
            for row in reader:
                writer.writerow(row)
                
        # remove the original CSV file and rename the modified CSV file to the original filename
        os.remove(os.path.join(directory, file))
        os.rename(os.path.join(directory, new_filename), os.path.join(directory, file))


In [24]:
#find_lowest_price from csvs
def find_lowest_price(result_name, result_column_name='result_name', price_column_name='latest_captured_price_usd', date_column_name='latest_captured_timestamp'):
    lowest_price_NZD = None
    lowest_price_USD = None
    lowest_date = None
    search_name = None
    csv_dir = currentdir+'\\file'
    # get a list of all CSV files in the directory
    files = os.listdir(csv_dir)
    csv_files = [f for f in files if f.endswith('.csv')]

    # loop over each CSV file
    for file in csv_files:

        # open the CSV file and create a CSV reader object
        with open(os.path.join(csv_dir, file), 'r') as f:
            reader = csv.DictReader(f)

            # loop over each row in the CSV file
            for row in reader:             

                # check if the result name matches the input
                if result_name.lower() in row[result_column_name].lower():
                    #print(row)
                    # convert the price to an integer
                    if row[price_column_name] is not None:
                        price = float(row[price_column_name])

                        # update the lowest price if it is None or greater than the current price
                        if lowest_price_USD is None or price < lowest_price_USD:
                            lowest_price_NZD = round(price * USD_NZD,2)
                            lowest_price_USD = round(price,2)
                            lowest_date = row[date_column_name]
                            search_name = row[result_column_name]

    return search_name,'NZD'+str(lowest_price_NZD),'USD'+str(lowest_price_USD), lowest_date


In [27]:
find_lowest_price('World Of Final Fantasy Maxima')

('World Of Final Fantasy Maxima', 'NZD11.86', 'USD7.32', '04/03/23 15:18:53')

In [26]:
#Run
#url_list = get_url(search_list)
#combined_list = get_result(url_list)
#combined_list = get_result_top_5(url_list)
#url_list = get_url(search_list)
url_list, name_list =get_url_v2(search_list)
filtered_list = get_result_matched(url_list,name_list)
#exchange_rate = get_currency('USD', 'NZD') #There is a bug on the api, so read it directly instead of via py pkg
gen_comparison(filtered_list,USD_NZD)


skip as bought 'CODE VEIN'
 already
skip as bought 'DRAGON BALL Z: KAKAROT'
 already
skip as bought 'SCARLET NEXUS'
 already
skip as bought 'Hades'
 already
skip as bought 'Sekiro'
 already
url list and name list are generated
skip Dragon Quest Xi Echoes Of An Elusive Age as no data return
skip Assassins Creed Valhalla The Siege Of Paris as no data return
skip Assassins Creed Valhalla Wrath Of The Druids as no data return
skip Assassins Creed Valhalla The Way Of The Berserker as no data return
skip Azure Striker Gunvolt Striker Pack as no data return
skip Azure Striker Gunvolt as no data return
skip Bayonetta Origins Cereza And The Lost Demon as no data return
skip Bayonetta 3 as no data return
skip Bayonetta 2 as no data return
skip Forza Horizon 4 Barrett Jackson Car Pack as no data return
skip Forza Horizon 4 1959 Cadillac Eldorado Biarritz Convertible as no data return
skip Forza Horizon 4 1963 Opel Kadett A as no data return
skip Forza Horizon 4 1977 Hoonigan Ford Gymkhana 10 F 15

C:\Users\303435\AppData\Local\Temp\ipykernel_22212\2354911962.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_cheaper[result_cheaper.select_dtypes(include=['number']).columns] = result_cheaper[result_cheaper.select_dtypes(include=['number']).columns].round(2) * exchange_rate


(                                   result_name release_yr  \
 4      Assassins Creed Valhalla Deluxe Edition       2022   
 12  Bayonetta Vanquish 10Th Anniversary Bundle       2022   
 28              Forza Horizon 5 Deluxe Edition       2018   
 38                              Nier Replicant       2021   
 41               Ori And The Will Of The Wisps       2020   
 65             Devil May Cry 5 Special Edition       2016   
 66                       Red Dead Redemption 2       2010   
 78               World Of Final Fantasy Maxima       2018   
 
     latest_captured_price_usd latest_captured_timestamp  \
 4                     46.9314         04/25/23 09:33:40   
 12                    27.3618         04/25/23 09:33:44   
 28                    69.2388         04/25/23 09:33:46   
 38                   102.0924         04/25/23 09:33:52   
 41                    11.2428         04/25/23 09:33:56   
 65                    20.1204         04/25/23 09:34:04   
 66                 